# Trabalho 1 - Inteligência Artificial

## Imports necessários

In [61]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

## Base de dados

### Matrícula: 2015100346
Dessa forma, devido ao final da matrícula ser 6, a base de dados será composta pelos 10 descritores de Fourier e os 7 descritores de Hu.

In [71]:
# Leitura dos dados
df = pd.read_csv('https://raw.githubusercontent.com/VitorBonella/PL-Dataset/main/dataset.csv',sep=";") 

# Transformando a coluna id no índice da tabela
df.set_index('id', inplace=True)

# Lista de descritores
FOURIER = ['df01', 'df02', 'df03', 'df04','df05', 'df06', 'df07', 'df08', 'df09', 'df10']
HU = ['i1', 'i2', 'i3', 'i4','i5', 'i6', 'i7']
HARALICK = ['probmax', 'energia', 'entropia', 'contraste','homogeneidade', 'correlacao']

# Descritores que serão usados nesse trabalho
dataset = df[FOURIER + HU] 

# Transformação dos dados de string para float devido ao uso da vírgula ao invés do ponto
dataset = dataset.apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

# Criação das classes baseada no tipo da lâmpada e na potência
classes = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

# Adiciona a classe ao data frame da base de dados
# dataset['classe'] = df['tipo_lampada'].str.replace(" ", "") + df['potencia'].astype(str) 

# Define a base de dados e as classes target
dataset_X = dataset
dataset_Y = classes

# from sklearn import datasets
# dataset = datasets.load_breast_cancer()
# dataset_X = dataset.data
# dataset_Y = dataset.target

print(classes)
dataset.head(2)


id
355    metalica400
356    metalica400
357    metalica400
358    metalica400
359    metalica400
          ...     
656    metalica250
657    metalica250
658    metalica250
659    metalica250
660    metalica250
Length: 297, dtype: object


,probmax,energia,entropia,contraste,homogeneidade,correlacao
id,,,,,,
355,0.890374,3.170342,6.571618,-30.712990,3.763049,0.000304
356,0.874335,3.056052,7.561434,-37.105195,3.722622,0.000269


# Cálculo dos resultados

In [63]:
def classification_report(scores):
    print(f'\nMédia: {scores.mean():.5f}, Desvio Padrão: {scores.std():.5f}')

    inf, sup = stats.norm.interval(0.95, loc=scores.mean(), 
                               scale=scores.std()/np.sqrt(len(scores)))
    
    print(f'Intervalo de confiança (95%): [{inf:.5f},{sup:.5f}]')

# ZeroR (ZR)

In [64]:
zR = DummyClassifier()

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', zR)])

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_zR = cross_val_score(pipeline, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_zR)

classification_report(scores_zR)

[0.16666667 0.13333333 0.16666667 0.16666667 0.16666667 0.16666667
 0.16666667 0.17241379 0.17241379 0.17241379 0.16666667 0.13333333
 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.17241379
 0.17241379 0.17241379 0.16666667 0.13333333 0.16666667 0.16666667
 0.16666667 0.16666667 0.16666667 0.17241379 0.17241379 0.17241379]

Média: 0.16506, Desvio Padrão: 0.01088
Intervalo de confiança (95%): [0.16116,0.16895]


# Bagging (BA)

In [65]:
grade = {'estimator__n_estimators':[3, 9, 15, 21]}

# TODO
# Talvez usar um estimador diferente no final do ensemble
# Opções: Decision Tree, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machines (SVM)
bg = BaggingClassifier(estimator=GaussianNB(), random_state=0)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', bg)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv=4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_BA = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_BA)

classification_report(scores_BA)

[0.66666667 0.36666667 0.53333333 0.33333333 0.4        0.36666667
 0.23333333 0.5862069  0.4137931  0.51724138 0.5        0.4
 0.46666667 0.53333333 0.4        0.46666667 0.43333333 0.31034483
 0.34482759 0.4137931  0.53333333 0.43333333 0.36666667 0.43333333
 0.36666667 0.36666667 0.5        0.48275862 0.48275862 0.4137931 ]

Média: 0.43552, Desvio Padrão: 0.08763
Intervalo de confiança (95%): [0.40416,0.46687]


# AdaBoost (AB)

In [66]:
grade = {'estimator__n_estimators':[3, 9, 15, 21]}

# TODO
# Talvez usar um estimador diferente no final do ensemble
# Opções: Decision Tree, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machines (SVM)
adb = AdaBoostClassifier(estimator=GaussianNB(), random_state=0)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', adb)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv=4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_AB = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_AB)

classification_report(scores_AB)

[0.2        0.33333333 0.4        0.23333333 0.5        0.26666667
 0.33333333 0.4137931  0.31034483 0.4137931  0.36666667 0.4
 0.33333333 0.3        0.36666667 0.2        0.4        0.24137931
 0.37931034 0.37931034 0.5        0.43333333 0.5        0.5
 0.4        0.4        0.36666667 0.37931034 0.4137931  0.27586207]

Média: 0.36467, Desvio Padrão: 0.08294
Intervalo de confiança (95%): [0.33500,0.39435]


# RandomForest (RF)

In [67]:
grade = {'randomForest__n_estimators': [3, 9, 15, 21]}

rF = RandomForestClassifier()

pipeline = Pipeline([('transformer', StandardScaler()), ('randomForest', rF)])

gs = GridSearchCV(estimator=pipeline, param_grid=grade, scoring='accuracy', cv = 4)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=36851234)

scores_RF = cross_val_score(gs, dataset_X, dataset_Y, scoring='accuracy', cv = rkf)

print(scores_RF)

classification_report(scores_RF)


[0.8        0.33333333 0.7        0.6        0.5        0.66666667
 0.56666667 0.62068966 0.62068966 0.68965517 0.4        0.43333333
 0.73333333 0.63333333 0.56666667 0.6        0.56666667 0.65517241
 0.55172414 0.68965517 0.53333333 0.56666667 0.63333333 0.5
 0.53333333 0.5        0.66666667 0.44827586 0.51724138 0.51724138]

Média: 0.57812, Desvio Padrão: 0.10096
Intervalo de confiança (95%): [0.54200,0.61425]


# Heterogeneous Pooling (HP)

In [68]:
from sklearn.utils import resample
from sklearn.utils.validation import check_X_y
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import numpy as np
from sklearn.base import BaseEstimator

class HeterogeneousPoolingClassifier(BaseEstimator):
    def __init__(self, n_samples):
        super().__init__()
        self.n_samples = n_samples
        self.classifiers = []

    def fit(self, x_train, y_train):
        x_train, y_train = check_X_y(x_train, y_train)
        classes = np.unique(y_train)
        self.class_order = self._get_class_order(y_train)

        for i in range(self.n_samples):
            # First iteration uses the original base for training
            if i == 0:
                current_x_train, current_y_train = x_train, y_train
            else:
                # Resample the training data with replacement
                current_x_train, current_y_train = resample(x_train, y_train, replace=True, random_state=i-1)

            dt_classifier = DecisionTreeClassifier()
            dt_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(dt_classifier)

            nb_classifier = GaussianNB()
            nb_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(nb_classifier)

            knn_classifier = KNeighborsClassifier()
            knn_classifier.fit(current_x_train, current_y_train)
            self.classifiers.append(knn_classifier)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.class_order)))

        for i in enumerate(self.class_order):
            for classifier in enumerate(self.classifiers[i::len(self.class_order)]):
                # Make predictions using each classifier for the current class
                predictions[:, i] += classifier.predict(X)

        final_predictions = []
        for pred_row in predictions:
            class_votes = Counter(pred_row)
            max_vote = max(class_votes.values())
            tie_classes = [class_label for class_label, votes in class_votes.items() if votes == max_vote]

            if len(tie_classes) > 1:
                # If multiple classes have the same highest vote, choose the most frequent class in the training data
                tie_class_counts = Counter(y)
                max_tie_vote = max([tie_class_counts[class_label] for class_label in tie_classes])
                most_frequent_tie_classes = [class_label for class_label in tie_classes if tie_class_counts[class_label] == max_tie_vote]
                final_predictions.append(max(most_frequent_tie_classes, key=lambda x: np.count_nonzero(self.class_order == x)))
            else:
                final_predictions.append(tie_classes[0])

        return np.array(final_predictions)

    def _get_class_order(self, y):
        class_counts = Counter(y)
        sorted_classes = sorted(class_counts, key=class_counts.get, reverse=True)
        return np.array(sorted_classes)


### Hp

O pseudo código a seguir mostra como o HP é obtido a partir de uma base de dados de treino:

- Obter e armazenar a ordenação das classes de acordo com a ocorrência nos exemplos na
base de treino (ordenar decrescentemente da mais frequente para a menos frequente)
- Para cada um dos n_samples faça
    - Se for a primeira iteração então
        - Usar a base original para treino dos classificadores
    - Senão
        - Montar uma base de treino de mesmo tamanho da original coletando aleatoriamente exemplos da base original com reposição
    - Fim-se
    - Treinar os classificadores NN, NB, DT na base de treino corrente e incluí-los no combinado de classificadores
- Fim-para

### Classificação final

O pseudo código seguinte mostra como o combinado HP é usado para classificar um exemplo
da base de dados de teste:

- Para cada um dos classificadores individuais do combinado faça
    - Obter a classificação do exemplo usando o classificador individual e armazenar a classe selecionada
- Fim-para
- Contar quantas vezes cada classe foi selecionada e obter a(s) mais votada(s)
- Se mais de uma classe for a mais votada então
    - Retornar a classe mais votada mais frequente na base de treino dentre as que empataram
- Senão
    - Retornar a classe mais votada
- Fim-se

In [69]:
classification_report(scores_zR)
classification_report(scores_BA)
classification_report(scores_AB)
classification_report(scores_RF)


Média: 0.16506, Desvio Padrão: 0.01088
Intervalo de confiança (95%): [0.16116,0.16895]

Média: 0.43552, Desvio Padrão: 0.08763
Intervalo de confiança (95%): [0.40416,0.46687]

Média: 0.36467, Desvio Padrão: 0.08294
Intervalo de confiança (95%): [0.33500,0.39435]

Média: 0.57812, Desvio Padrão: 0.10096
Intervalo de confiança (95%): [0.54200,0.61425]
